# Estudo de caso: Bacia do Paranapanema

## Imports

In [692]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

from mogestpy.quality import buwo
from mogestpy.quantity.Hydrological import SMAP
from spotpy.objectivefunctions import rmse, nashsutcliffe, pbias, kge

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

## Inicialização do modelo de bacias

### Dados com os parâmetros SMAP - Buildup/Washoff - Muskingum

In [693]:
df_data = pd.read_excel('case.xlsx')

### Dados meteorológicos

In [694]:
df_prec = pd.read_excel('prec.xlsx')
df_etp = pd.read_excel('etp.xlsx')

Carrega as informações da bacia para o modelo SMAP

In [695]:
basins = []
for i in range(8):
    basin = SMAP.SMAP.Basin(AD=df_data['AD'][i],
                            Str=df_data['Sat'][i],
                            Capc=df_data['Capc'][i],
                            Crec=df_data['Crec'][i],
                            TUin=df_data['Tuin'][i],
                            EBin=df_data['Ebin'][i],
                            k2t=df_data['k2t'][i],
                            kkt=df_data['kkt'][i],
                            Ai=df_data['Ai'][i])
    basins.append(basin)

Verificação dos parâmetros das subbacias 89 e 88

In [696]:
print(basins[7])

SMAP Basin Object:
  Str = 500,
  Crec = 0.005,
  Capc = 0.3,
  kkt = 115,
  k2t = 5,
  Ai = 2,
  TUin = 0.09,
  EBin = 0.07099238269887274,
  AD = 3201.242645


In [697]:
basins[7].IsValid()

True

Códigos das ottobacias de nível 5 do estudo de caso

In [698]:
codes = [86492, 86493, 86494, 86495, 86496, 86497, 86498, 86499]

Criando pontos com precipitação e evapotranspiração das bacias

In [699]:
points = []
for i in range(8):
    point = SMAP.SMAP.Point(df_prec[codes[i]], df_etp[codes[i]])
    points.append(point)

In [700]:
models = []
for i in range(8):
    model = SMAP.SMAP(points[i], basins[i])
    models.append(model)

In [701]:
for model in models:
    model.RunModel()

Criando DataFrames (Tabelas) de vazão incremental (cada ottobacia) e escoamento superficial direto

In [702]:
df_Qinc = pd.DataFrame()
df_Qd = pd.DataFrame()

In [703]:
for i in range(8):
    df_Qinc[codes[i]] = models[i].Q
    df_Qd[codes[i]] = models[i].Qd


Criando coluna com dados da estação de monitoramento a montante da bacia 86497 (das quais as bacias 86498 e 86499 contribuem)

In [704]:
df_Qinc['86498+86499'] = df_Qinc[86498] + df_Qinc[86499]

Carregando dados de vazão observada para a estação de monitoramento utilizada

In [705]:
df_obs = pd.read_excel('qobs.xlsx')

In [706]:
df_Qinc['Qobs'] = df_obs['Qobs']

Gráfico de vazão na estação de monitoramento 2011 - 2012

In [707]:
df_Qinc['Qobs'] = df_obs['Qobs'].replace(0, float('nan')).dropna()
fig = df_Qinc[['86498+86499', 'Qobs']].plot()

fig.update_layout(legend_title='Legenda', legend=dict(y=1.1, orientation='h', itemsizing='constant'), font_family='CMU Serif', font=dict(size=12))
fig.update_xaxes(title_text='Tempo (dias)', title_font=dict(size=12))
fig.update_yaxes(title_text='Vazão (m³/s)', title_font=dict(size=12))
fig.update_layout(
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=700,
    height=500
)

fig.show()

Calculando indicadores para o período de 2012

In [708]:
rmse_val = rmse(df_Qinc['Qobs'][365:], df_Qinc['86498+86499'][365:])
pbias_val = pbias(df_Qinc['Qobs'][365:], df_Qinc['86498+86499'][365:])
nashsutcliffe_val = nashsutcliffe(df_Qinc['Qobs'][365:], df_Qinc['86498+86499'][365:])
kge_val = kge(df_Qinc['Qobs'][365:], df_Qinc['86498+86499'][365:])

print(f"RMSE: {rmse_val:.1f}")
print(f"PBIAS: {pbias_val:.1f}%")
print(f"NSE: {nashsutcliffe_val:.3f}")
print(f"KGE: {kge_val:.3f}")



RMSE: 26.5
PBIAS: -8.1%
NSE: 0.619
KGE: 0.804


Verificação dos valores das parcelas de escoamento base e escoamento superficial direto

In [709]:
df_test = pd.DataFrame()

Verificação da vazão superficial direta e de base do modelo SMAP

In [710]:
df_test['Qb'] = models[0].Qb
df_test['Qd'] = models[0].Qd
df_test['Qt'] = models[0].Q

fig = df_test.plot()
fig.update_layout(legend_title='Legenda', legend=dict(y=1.1, orientation='h', itemsizing='constant'), font_family='CMU Serif', font=dict(size=12))
fig.update_xaxes(title_text='Tempo (dias)', title_font=dict(size=12))
fig.update_yaxes(title_text='Vazão (m³/s)', title_font=dict(size=12))
fig.update_layout(
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=800,
    height=600
)

fig.show()


Configurando o modelo de acumulo e lavagem em bacias (Buildup/Washoff)

In [711]:
loadgens = []

for i in range(8):
    loadgen = buwo.BuildUpWashoff(landuse_name='',
                                  Bmax=df_data['Bmax'][i],
                                  Nb=df_data['Nb'][i],
                                  Kb=df_data['Kb'][i],
                                  threshold_flow=0.02,
                                  Nw=df_data['Nw'][i],
                                  Kw=df_data['Kw'][i],
                                  BuMethod=2,
                                  WoMethod=1,
                                  timestep_h=24,
                                  initial_buildup=1,
                                  area=df_data['AD'][i],
                                  area_fraction=1,
                                  surface_flow=df_Qd[codes[i]]/df_data['AD'][i]*86.4)
    loadgens.append(loadgen)

Convertendo dados de vazão de (**m³/s**) para (**mm/dia**)

In [712]:
df_Qmm = pd.DataFrame()

for i in range(8):
    df_Qmm[codes[i]] = df_Qd[codes[i]]/df_data['AD'][i]*86.4
    
df_Qmm

,86492,86493,86494,86495,86496,86497,86498,86499
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000725,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000632,0.012828,0.063548,0.063534,0.000000,0.000030,0.001985,0.000000
3,0.000550,0.029445,0.139832,0.139692,0.015715,0.042964,0.085115,0.027260
4,0.029468,0.039775,0.126091,0.125956,0.013681,0.047790,0.122848,0.035757
...,...,...,...,...,...,...,...,...
726,1.508263,0.494187,0.519475,0.504003,0.228099,0.909079,0.256576,0.168080
727,3.577773,0.942419,0.476709,0.462386,0.237679,1.000118,0.261244,0.176511
728,3.529405,1.384246,1.128292,1.092861,0.211089,2.578774,0.227426,0.242259
729,3.105177,1.205457,0.982235,0.951391,0.200630,2.244954,0.311057,0.397081


Gráfico de Escoamento Superficial Direto para toda a região do estudo

In [713]:
fig = df_Qmm.plot()
fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Vazão (mm/dia)',
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=700,
    height=500
)
fig.show()


In [714]:
df_Qmm.to_excel('Vazao_mm.xlsx')

In [715]:
for loadgen in loadgens:
    loadgen.Process(verbose=False)

In [716]:
df_Wo = pd.DataFrame()
for i in range(8):
    df_Wo[codes[i]] = loadgens[i].Washoff

Gráfico de Washoff (Lavagem) em massa DBO

In [717]:
fig = df_Wo.plot()
fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Carga (kg/dia)',
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=800,
    height=600
)
fig.show()


In [718]:
try:
    df_Wo.to_excel('Cargas.xlsx')
    print("File saved successfully!")
except Exception as e:
    print(f"Error saving file: {e}")


Error saving file: [Errno 13] Permission denied: 'Cargas.xlsx'


In [727]:
df_WoC = pd.DataFrame()
for i in range(8):
    df_WoC[codes[i]] = df_Wo[codes[i]]/df_data['AD'][i]*1000/86.4

Polutograma (mg/L) DBO

In [720]:
fig = df_WoC[365:].plot()
fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Carga (mg/L)',
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=800,
    height=600
)
fig.show()


In [721]:
df_WoC.to_excel('Cargas_mg-L.xlsx')

In [728]:
df_Bu = pd.DataFrame()  

for i in range(8):
    df_Bu[codes[i]] = loadgens[i].BuildUp

In [723]:
df_Bu_ton = df_Bu.div(1e3) # conversão para ton

### Gráfico de Buildup para o ano de 2012

x = tempo (dias); x0 = 01/01/2012

y = Buildup (kg/dia)

In [724]:
fig = df_Bu_ton[:365].plot.bar()

fig.update_yaxes(title_text='Carga (ton/dia)', secondary_y=False)
fig.update_layout(xaxis_title='Tempo (dias)', font=dict(size=12), legend=dict(y=1.1, orientation='h', itemsizing='constant'))
fig.update_layout(
    font=dict(size=12),
    width=1000,
    height=600
)

fig.show()

In [725]:
import locale
from plotly.subplots import make_subplots

# set locale to PT-BR
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

df_estrela = pd.DataFrame()
df_estrela['86498+86499'] = df_Bu_ton[86498] + df_Bu_ton[86499]
df_estrela = df_estrela.iloc[:365]
df_estrela['subtotal'] = df_estrela['86498+86499'].cumsum()
df_estrela.index = pd.date_range(start='2012-01-01', periods=len(df_estrela), freq='D')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_estrela.index, y=df_estrela['86498+86499'], name='Carga (ton/dia)', marker_color='blue'), secondary_y=False)
fig.add_trace(go.Scatter(x=df_estrela.index, y=df_estrela['subtotal'], name='Carga Acumulada (ton)', line=dict(color='red', width=2)), secondary_y=True)

fig.update_layout(title='Ponto de Controle - Montante da Ottobacia 86497', xaxis_title='Tempo (dias)', font=dict(size=12), legend=dict(y=1.1, orientation='h', itemsizing='constant'))
fig.update_yaxes(title_text='Carga (ton/dia)', secondary_y=False)
fig.update_yaxes(title_text='Carga Acumulada (ton)', secondary_y=True)
fig.update_layout(
    font=dict(size=12),
    width=800,
    height=600
)

fig.show()



In [726]:
df_Bu.to_excel('BuildUp.xlsx')